# **Primera Etapa de Predicción de Aterrizaje del Falcon 9 de Space X**


## Extracción de Datos de los Registros de Lanzamiento del Falcon 9 y el Falcon Heavy de Wikipedia


Tiempo necesario estimado: **40** minutos


En esta práctica, realizarás una extracción de datos de web para recopilar registros históricos de lanzamiento del Falcon 9 de una página de Wikipedia titulada `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


El Falcon 9 aterrizando con éxito:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Aquí se muestran varios ejemplos de aterrizajes fallidos:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


Más específicamente, los registros de lanzamiento están almacenados en una tabla HTML que se muestra a continuación:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objetivos

Extraer datos web de los registros de lanzamiento del Falcon 9 con `BeautifulSoup`:

*   Extrae una tabla HTML de los registros de lanzamiento del Falcon 9 de Wikipedia
*   Analiza la tabla y conviértela en un data frame de Pandas


Primero vamos a importar los paquetes requeridos para esta práctica


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

y proporcionaremos algunas funciones de ayuda para que proceses la tabla HTML extraída de la web


In [ ]:
def date_time(table_cells):
    """
    Esta función devuelve los datos y la hora de la celda de la tabla HTML
    Input: el elemento de una celda de datos de tabla extrae una fila adicional
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Esta función devuelve la versión de refuerzo de la celda de la tabla HTML 
    Input: el elemento de una celda de datos de tabla extrae una fila adicional
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    Esta función devuelve el estado de aterrizaje de la celda de la tabla HTML 
    Input: el elemento de una celda de datos de tabla extrae una fila adicional
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    Esta función devuelve el estado de aterrizaje de la celda de la tabla HTML
    Input: el elemento de una celda de datos de tabla extrae una fila adicional
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filtramos los dígitos y los nombres vacíos
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


Para mantener la coherencia de las tareas de la práctica, se te pedirá que extraigas los datos de una foto de  `List of Falcon 9 and Falcon Heavy launches` Wikipage actualizada el
`9th June 2021`


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

A continuación, solicita la página HTML de la URL anterior y obtenga un objeto de `respuesta`


### TAREA 1: Solicita la página de Wikipedia del Lanamiento del Falcon9 a través de su URL


Primero, realicemos un método HTTP GET para solicitar la página HTML de lanzamiento de Falcon 9, como una respuesta HTTP.


In [ ]:
# usa el método request.get() con el static_url proporcionado
# asignar la respuesta a un objeto
response=requests.get(static_url)

Crea un objeto `BeautifulSoup` a partir de la `respuesta` HTML


In [ ]:
# Usa BeautifulSoup() para crear un objeto BeautifulSoup a partir de un contenido del texto de respuesta
soup= BeautifulSoup(response.text)

Imprime el título de la página para verificar si el objeto `BeautifulSoup` se creó correctamente

In [ ]:
# Usa el atributo soup.title
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TAREA 2: Extrae todos los nombres de la columna/variable del encabezado de la tabla HTML


A continuación, queremos recopilar todos los nombres de columna relevantes del encabezado de la tabla HTML


Intentemos encontrar todas las tablas en la página wiki primero. Si necesitas refrescar tu memoria sobre `BeautifulSoup`, consulta el enlace de referencia externo hacia el final de esta práctica


In [ ]:
# Usa la función find_all en el objeto BeautifulSoup, con el tipo de elemento `table`
# Asigna el resultado a una lista llamada `html_tables`
html_tables=soup.find_all('table')

A partir de la tercera tabla, nuestra tabla de destino contiene los registros de lanzamiento reales.


In [ ]:
# Imprimamos la tercera tabla y comprobemos su contenido.
first_launch_table = html_tables[2]

Deberías poder ver los nombres de las columnas incrustados en los elementos del encabezado de la tabla `<th>` de la siguiente manera:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


A continuación, solo necesitamos iterar a través de los elementos `<th>` y aplicar el `extract_column_from_header()` proporcionado para extraer el nombre de la columna uno por uno


In [ ]:
column_names = []
find_th=first_launch_table.find_all('th')
for th in find_th:
    th_name=extract_column_from_header(th)
    column_names.append(th_name)

# Aplica la función find_all() con el elemento `th` en first_launch_table
# Itera cada elemento th y aplica el extract_column_from_header() proporcionado para obtener un nombre de columna
# Agrega el nombre de la columna Non-empty (`si el nombre no es None y len(name)> 0`) en una lista llamada column_names


Comprueba los nombres de columnas extraídos


In [ ]:
print(column_names)

['Flight No.', 'Date and time ( )', '', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', '', None, None, None, None, None, None, None]


## TAREA 3: Crea un data frame analizando las tablas HTML de lanzamiento


Crearemos un diccionario vacío con claves de los nombres de las columnas extraídas en la tarea anterior. Más tarde, este diccionario será convertido en un dataframe de Pandas


In [ ]:
launch_dict= dict.fromkeys(column_names)

# Elimina una columna irrelevante
del launch_dict['Date and time ( )']

# Inicialicemos el launch_dict con cada valor para que sea una lista vacía
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Añade algunas columnas nuevas
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

A continuación, solo necesitamos completar `launch_dict` con los registros de lanzamiento extraídos de las filas de la tabla.


Por lo general, es probable que las tablas HTML en las páginas Wiki contengan anotaciones inesperadas y otros tipos de ruidos, como enlaces de referencia `B0004.1[8]`, valores faltantes `N/A [e]`, formato inconsistente, etc.


Para simplificar el proceso de análisis, hemos proporcionado un fragmento de código incompleto abajo para ayudarte a completar el `launch_dict`. Completa el siguiente fragmento de código con los TODO o puedes optar por escribir tu propia lógica para analizar todas las tablas de lanzamiento:


In [ ]:
extracted_row = 0
#Extrae cada tabla 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # obtén la fila de la tabla 
    for rows in table.find_all("tr"):
        #Comprueba si el encabezado de la primera tabla es el número correspondiente para lanzar un número
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #obtén el elemento de la tabla 
        row=rows.find_all('td')
        #si es un número guardar celdas en un diccionario
        if flag:
            extracted_row += 1
            # Valor del número de vuelo
            # TODO: Agrega flight_number en el launch_dict con la clave `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Valor de la fecha
            # TODO: Agrega date en launch_dict con la clave `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Valor de la hora
            # TODO: Agrega time en launch_dict con la clave `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Versión de refuerzo
            # TODO: Agrega bv en launch_dict con la clave `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            #print(bv)
            
            # Sitio de lanzamiento
            # TODO: Agrega bv en launch_dict con la clave `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            
            # Carga útil
            # TODO: Agrega payload en launch_dict con la clave `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            
            # Masa de la carga útil
            # TODO: Agrega payload_mass en launch_dict con la clave `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            
            # Órbita
            # TODO: Agrega orbit en launch_dict con la clave `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)
            
            # Cliente
            # TODO: Agrega customer en launch_dict con la clave `Customer`
            if (row[6].a is not None):
                customer = row[6].a.string
            else:
                customer = row[6].string
            launch_dict['Customer'].append(customer)
            #print(customer)
            
            # Resultado del lanzamiento
            # TODO: Agrega launch_outcome en launch_dict con la clave `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Aterrizaje de refuerzo
            # TODO: Agrega booster_landing en launch_dict con la clave `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)
            

Después de completar los valores de registro de lanzamiento analizados en `launch_dict`, puedes crear un dataframe a partir de él.


In [ ]:
df=pd.DataFrame(launch_dict)

LIMPIEZA DE DATOS

In [ ]:
df.drop(df.columns[1], axis=1)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,NaN,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,None,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,None,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,None,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,None,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,None,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,None,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,None,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,None,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,None,F9 B5B1067.1,Success,3 June 2021,17:29


Ahora podemos exportarlo a un <b>CSV</b> para la siguiente sección, para que las respuestas sean consistentes y en caso de que tengas dificultades para terminar esta práctica.

Las siguientes prácticas utilizarán un conjunto de datos proporcionado para que cada práctica sea independiente.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>
